In [5]:
import gmsh

# Define the file paths
step_file_path = "torus.stp"
output_mesh_path = "mesh.inp"  # Change the extension to '.inp'

# Initialize Gmsh
gmsh.initialize()

# Create a Gmsh model
gmsh.model.add("imported_model")

# Import the STEP file using Gmsh
gmsh.merge(step_file_path)
a = gmsh.merge(step_file_path)
# Generate the mesh using Gmsh
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.1)  # Adjust mesh parameters
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.01)  # Adjust mesh parameters
gmsh.option.setNumber("Mesh.ElementOrder", 2)  # Element order (1 for linear, 2 for quadratic)
gmsh.option.setNumber("Mesh.Optimize", 1)  # Mesh optimization
gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)  # Subdivision algorithm (1 for Delaunay)
gmsh.model.mesh.generate(2)

# Save the mesh to an .inp file
gmsh.write(output_mesh_path)

# Finalize Gmsh
gmsh.finalize()

print(f"Mesh saved to {output_mesh_path}")


Mesh saved to mesh.inp


In [4]:
import numpy as np

def finite_element_analysis(L, w, h, E, I):
    """
    Performing a finite element analysis of a cantilever beam with a uniformly distributed load.

    Args:
        L: Length of the beam in meters.
        w: Uniformly distributed load in Newtons per meter.
        h: Height of the beam in meters.
        E: Young's modulus of the beam material in Pascals.
        I: Moment of inertia of the beam cross-section in meters^4.

    Returns:
        A tuple of the following:
            * deflections: A numpy array of the deflections at each node in the beam.
            * bending_moments: A numpy array of the bending moments at each node in the beam.
            * shear_forces: A numpy array of the shear forces at each node in the beam.
    """

    # Calculate element stiffness matrix.
    element_stiffness_matrix = np.array([[12 * E * I / L**3, 6 * E * I / L**2],
                                         [6 * E * I / L**2, 4 * E * I / L]])

    # Assemble the global stiffness matrix.
    global_stiffness_matrix = np.zeros((2, 2))
    global_stiffness_matrix[0:2, 0:2] += element_stiffness_matrix

    # Define the load vector.
    load_vector = np.array([w * L**2 / 2, w * L])

    try:
        # Solve the system of equations.
        deflections = np.linalg.solve(global_stiffness_matrix, load_vector)

        # Calculate the bending moments and shear forces.
        bending_moments = np.array([6 * E * I * (deflections[1] - deflections[0]) / L, 6 * E * I * deflections[1] / L])
        shear_forces = np.array([w * L / 2 - bending_moments[0], w * L - bending_moments[1]])

        return deflections, bending_moments, shear_forces

    except np.linalg.LinAlgError:
        # Handle cases where the matrix cannot be inverted.
        raise ValueError("Unable to solve the system. Check input values for validity.")

# Define the beam properties.
L = 1.0  # meters
w = 100.0  # Newtons per meter
h = 0.1  # meters
E = 200e9  # Pascals
I = h**3 / 12  # meters^4

# Perform the finite element analysis.
deflections, bending_moments, shear_forces = finite_element_analysis(L, w, h, E, I)

# Print the results.
print("Deflections:", deflections)
print("Bending Moments:", bending_moments)
print("Shear Forces:", shear_forces)


Deflections: [-2.0e-06  4.5e-06]
Bending Moments: [650. 450.]
Shear Forces: [-600. -350.]


In [8]:
import numpy as np

def stiffness_matrix(L, E, I):
    """
    Calculate the stiffness matrix for an element of a cantilever beam.

    Args:
        L: Length of the element in meters.
        E: Young's modulus of the beam material in Pascals.
        I: Moment of inertia of the beam cross-section in meters^4.

    Returns:
        The element stiffness matrix as a numpy array.
    """
    k = np.array([[12 * E * I / L**3, 6 * E * I / L**2, -12 * E * I / L**3, 6 * E * I / L**2],
                  [6 * E * I / L**2, 4 * E * I / L, -6 * E * I / L**2, 2 * E * I / L],
                  [-12 * E * I / L**3, -6 * E * I / L**2, 12 * E * I / L**3, -6 * E * I / L**2],
                  [6 * E * I / L**2, 2 * E * I / L, -6 * E * I / L**2, 4 * E * I / L]])
    return k

def finite_element_analysis(L, num_elements, w, h, E):
    """
    Performs a finite element analysis of a cantilever beam with a uniformly distributed load.

    Args:
        L: Length of the beam in meters.
        num_elements: Number of elements to discretize the beam.
        w: Uniformly distributed load in Newtons per meter.
        h: Height of the beam in meters.
        E: Young's modulus of the beam material in Pascals.

    Returns:
        A tuple of the following:
            * deflections: A numpy array of the deflections at each node in the beam.
            * moments: A numpy array of the bending moments at each node in the beam.
            * shear_forces: A numpy array of the shear forces at each node in the beam.
    """
    # Calculate the moment of inertia of the I-section.
    I_section = 2 * h**3 / 12 + h * (h / 2)**2

    # Initialize the global stiffness matrix and load vector.
    num_nodes = num_elements + 1
    K_global = np.zeros((2 * num_nodes, 2 * num_nodes))
    F = np.zeros(2 * num_nodes)

    # Calculate the length of each element.
    element_length = L / num_elements

    # Assemble the global stiffness matrix and load vector.
    for i in range(num_elements):
        # Calculate the element stiffness matrix for the current element.
        k_element = stiffness_matrix(element_length, E, I_section)

        # Add the contribution of the current element to the global stiffness matrix.
        node1 = i
        node2 = i + 1
        indices = [2 * node1, 2 * node1 + 1, 2 * node2, 2 * node2 + 1]
        for r in range(4):
            for c in range(4):
                K_global[indices[r], indices[c]] += k_element[r, c]

        # Add the contribution of the UDL to the load vector.
        F[2 * node1 + 1] += -0.5 * w * element_length**2
        F[2 * node2 + 1] += -0.5 * w * element_length**2

    # Apply boundary conditions (zero displacement at the fixed end).
    K_reduced = K_global[2:, 2:]
    F_reduced = F[2:]

    # Solve the system of equations for displacements using NumPy's linalg.solve.
    displacements = np.linalg.solve(K_reduced, F_reduced)

    # Combine displacements with boundary conditions.
    deflections = np.insert(displacements, 0, [0.0, 0.0])

    # Calculate moments and shear forces.
    moments = np.zeros(num_nodes)
    shear_forces = np.zeros(num_nodes)
    for i in range(num_nodes - 1):
        node1 = i
        node2 = i + 1
        indices = [2 * node1, 2 * node1 + 1, 2 * node2, 2 * node2 + 1]
        element_displacements = deflections[indices]
        k_element = stiffness_matrix(element_length, E, I_section)
        element_moments = np.dot(k_element[1:3, :], element_displacements)
        moments[node1] += element_moments[0]
        moments[node2] += element_moments[1]
        shear_forces[node1] += -element_displacements[1]
        shear_forces[node2] += -element_displacements[3]

    return deflections, moments, shear_forces

# Define the beam properties.
L = 1.0  # meters
w = 100.0  # Newtons per meter
h = 0.1  # meters
E = 200e9  # Pascals

# Calculate the number of elements such that the total number of nodes matches the length of the arrays.
num_elements = int(L / (h * 2))  # Each element spans a height of h/2

# Perform the finite element analysis.
deflections, moments, shear_forces = finite_element_analysis(L, num_elements, w, h, E)

# Print the results.
print("Node\tDeflection (m)\tBending Moment (Nm)\tShear Force (N)")
for i in range(len(deflections)):
    print(f"{i}\t{deflections[i]:.4f}\t\t{moments[i]:.4f}\t\t\t{shear_forces[i]:.4f}")


Node	Deflection (m)	Bending Moment (Nm)	Shear Force (N)
0	0.0000		18.0000			0.0000
1	0.0000		14.0000			0.0000
2	-0.0000		10.0000			0.0000
3	-0.0000		6.0000			0.0000
4	-0.0000		2.0000			0.0000
5	-0.0000		0.0000			0.0000


IndexError: index 6 is out of bounds for axis 0 with size 6